<a href="https://colab.research.google.com/github/seank1m/artificial-intelligence-for-robotics/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Past Sections from Assignment 1

In Assignment 2, we build upon the previous assignment and perform tasks related to regression problems.

---

First, we reload the dependencies from the previous assignment:

In [1]:
%pip install -q plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def plot_cloud(data=None, x='x', y='y', z='z', color=None, *, max_points=10_000, **kwargs):

    if max_points is not None and data is not None and len(data) > max_points:
        import sys
        print(f"Warning: too many points, trying to show only {max_points:,} points.", file=sys.stderr)
        skip = len(data) // max_points
        data = data[::skip]

    if isinstance(data, np.ndarray):
        m = data.shape[-1]
        if m == 3:
            x, y, z = data.T
        elif m == 4:
            x, y, z, color = data.T
        data_frame = None
    elif isinstance(data, pd.DataFrame):
        data_frame = data
    else:
        data_frame = None

    if (data_frame is not None
        and isinstance(color, str)
        and len(color) == 3
        and color not in data_frame.columns
        and all(c in data_frame.columns for c in color)
    ):
        update_color = data_frame[list(color)].to_numpy()
        color = None
    else:
        update_color = None

    fig = px.scatter_3d(
        data_frame,
        x=x, y=y, z=z,
        color=color,
        template=plot_cloud.template,
        **kwargs,
    )
    if update_color is not None:
        # has to be updated later because px.scatter_3d doesn't accept a matrix
        fig.data[0].marker.color = update_color

    return fig

plot_cloud.template = dict(
    layout=dict(
        margin=dict(
            # l=0, r=0,  # set left and right margin
            b=0, t=0,  # set bottom and top margin
        ),
        scene=dict(
            # xaxis_visible=False,  # hide axes
            # yaxis_visible=False,
            # zaxis_visible=False,
            aspectmode='data',  # set aspect ratio
        ),
    ),
    data=dict(
        scatter3d=[
            go.Scatter3d(
                marker_size=1,  # set default marker size
            ),
        ],
    ),
)

In [ ]:
import math
import numpy as np
import pandas as pd

n_clusters = 1000

We now use the `request` package to download `df_test` that contains the data required for the rest of the assignment. `df_test` might be slightly different from the one you got in Assignment 1 depending on the features you used for the classification (it does not mean that the result you got in Assignment 1 is wrong).

In [ ]:
import requests

# Shareable link
shareable_link = 'https://drive.google.com/file/d/1-5gZVjT5nIt4oAvTTVLk1zAmg9z55Nb1/view?usp=sharing'

# Extract file ID and create direct download URL
file_id = shareable_link.split('/')[-2]
url = f'https://drive.google.com/uc?export=download&id={file_id}'
print(f"Download URL: {url}")

# Download the file
output = 'df_test.pkl'
response = requests.get(url)
if response.status_code == 200:
    with open(output, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded as '{output}'")
else:
    print(f"Failed to download. Status code: {response.status_code}")

# Load the DataFrame (for pickle files)
df_test = pd.read_pickle(output)

# Show the first few rows
df_test.head()

# Assignment 2

Assignment 2 will continue on from Assignment 1, using the clustered and classified point cloud. We will use linear and non-linear regression models to resample the point cloud (the marking criteria is provided in brackets):

1. Apply linear regression to buildings (30%)
2. Answer the questions related to the linear regression (20%)
3. Apply the GP for ground estimation (30%)
4. Answer the questions related to the GP section (20%)

---


You are required to:
 - Model each cluster classified as *building* with a *linear regression* model.
 - Model every point classified as *terrain* with a *Gaussian process* model.

 > Begin by inserting Assignment 1 here. Note: by shift-clicking and then right-clicking, you can select multiple notebook cells, copy, and paste them across notebooks.

 > In the subsequent tasks, use the test data set `df_test` and group the points into **100 clusters**. Do not use the ground truth class `labels`, only the predicted values `pred`.


# 1. Linear Regression

As we are working in 3 dimensions, we are going to use 2 dimensions to estimate the 3rd. As such, There are 3 possible choices for the estimated variable: $x$, $y$, or $z$. Luckily, it is easy to know which model to select: the one with the lowest *mean squared error*.

> Define a function to perform linear regression in each of the 3 dimensions, and select the model with the best error.

> Apply this process to each cluster, replacing the points with a linear approximation.

> The model can be used at arbitrary locations, use `bounding_grid` to query each model on a regular grid instead of at the original (training) locations.

In [ ]:
# df_walls = df_test[df_test['pred'] == 'buildings']
positions = df_walls[['x','y','z']].to_numpy()
assignment = df_walls['cluster'].to_numpy()
print(positions)

In [ ]:
def bounding_grid(x, n=None):
    """
    Replaces a set of points by a similar number of points on a regular grid
    spanning the bounds of the original point set.

    x: numpy array, shape [n, d], n points in d dimensions.
    n: int (optional), minimum number of points to generate.
    """
    d = x.shape[-1]
    if n is None:
        n = len(x)
    mins = x.min(axis=0)
    maxs = x.max(axis=0)
    rngs = (maxs - mins)
    nums = np.ceil(rngs * np.power(n/rngs.prod(), 1/d)).astype(int)
    axes = [np.linspace(mn, mx, ns) for mn, mx, ns in zip(mins, maxs, nums)]
    grid = np.stack(np.meshgrid(*axes, indexing='ij'), axis=-1)
    return grid.reshape(nums.prod(), d)

In [ ]:
## TODO START

def lin_reg(points):
 ..............##TODO................

    return points


positions_lin = []
for i in range(n_clusters):
  ##TODO
     ...
    points_lin = lin_reg(points)
    positions_lin.append(points_lin)
positions_lin = np.concatenate(positions_lin)

plot_cloud(positions).show()
plot_cloud(positions_lin).show()

# 2. Linear regression Questions (In less than 100 words each):
- How shall we estimate the performance of the linear regression?
- What could we do if the the fitting of the plane was not good enough?
- How can we tell if the model is overfitting or underfitting our data?

# 3. Gaussian Process Regression

Similarly, Gaussian process models can be used in higher dimensions too. Model the ground with a GP, where the height is the estimated quantity.

> Perform Gaussian process regression on the height data.

> Note: we are no longer working with zero-mean data, and the kernel hyperparameters should be  chosen carefully.

> With the 2-dimensional input, the kernel function needs to change: it should take 2D vectors and return scalars.

> The training data should be considered noisy, with a standard deviation of 1cm (point cloud units are in meters).

> The model can be used at arbitrary locations, use `bounding_grid` to query the model on a regular grid instead of at the original (training) locations.

In [ ]:
df_ground = df_test[df_test['pred'] == 'terrain']
locations = df_ground[['x','y']].to_numpy()
heights = df_ground['z'].to_numpy()

In [ ]:
# See the mean and variance of the modelled property
mu = ... # TODO
sigma=...  # TODO
mu, sigma

In [ ]:
from numpy import exp, sqrt, sum
from numpy import diag, eye
from numpy.linalg import inv, solve

lengthscale =  # meter  # TODO

def kernel(x1, x2, sigma=sigma, lengthscale=lengthscale):
    return  ... # TODO

In [ ]:
## inputs
xd = ...
yd = ...
nd = ...
xq = ...

std_n =#TODO
Snn = #TODO

## evaluating the kernel
#TODO
Kdd = ...
Kqd = ...
Kdq = ...
Kqq = ...

## the GP equations
#TODO

mean_q = #TODO
cov_qq =#TODO
var_q  =   # TODO
std_q  =  #TODO

Plot the estimated height of the ground alongside with the uncertainty:

In [ ]:
# TODO
# plot the ground truth

# then plot the estimation with the uncertainty


# 4. GP Questions (In less than 100 words each):
- What is happening in the area where no points are rpovided to the GP (discuss the mean values and the uncertainty)?
- Which kernel we are using and why this is important?

